# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
from sqlalchemy import create_engine
import re
import numpy as np
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import confusion_matrix,classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'

In [2]:
# load data from database
engine = create_engine('sqlite:///InsertDatabaseName.db')
df = pd.read_sql_table("disaster_dataset_001" ,engine)
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [64]:
X = df.iloc[:,1]
Y = df.iloc[:,4:]

In [65]:
X.head()

0    Weather update - a cold front from Cuba that c...
1              Is the Hurricane over or is it not over
2                      Looking for someone but no name
3    UN reports Leogane 80-90 destroyed. Only Hospi...
4    says: west side of Haiti, rest of the country ...
Name: message, dtype: object

In [61]:
Y.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 2. Write a tokenization function to process your text data

In [66]:
def tokenize(text):
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    
    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)
        
    return clean_tokens

In [68]:
X1 = tokenize(X[1])

In [69]:
X1

['is', 'the', 'hurricane', 'over', 'or', 'is', 'it', 'not', 'over']

In [77]:
X2 = TfidfTransformer().fit_transform(X1)

TypeError: no supported conversion for types: (dtype('<U9'),)

In [73]:
X2

TfidfTransformer(norm=['is', 'the', 'hurricane', 'over', 'or', 'is', 'it', 'not', 'over'],
         smooth_idf=True, sublinear_tf=False, use_idf=True)

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [70]:
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf',MultiOutputClassifier(RandomForestClassifier()),
        )
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [71]:
X_train, X_test, y_train, y_test = train_test_split(X, Y)
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [25]:
y_pred = pipeline.predict(X_test)
y_pred

array([[1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ..., 
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [26]:
y_pred[:,0]

array([1, 1, 1, ..., 1, 1, 0])

In [27]:
Y.columns[0]

'related'

In [28]:
for i in range(len(Y.columns)):
    print(Y.columns[i])
    print(classification_report(y_pred[:,i], y_test.iloc[:,i]))

related
             precision    recall  f1-score   support

          0       0.34      0.60      0.43       869
          1       0.93      0.81      0.87      5673
          2       0.11      0.50      0.17        12

avg / total       0.85      0.78      0.81      6554

request
             precision    recall  f1-score   support

          0       0.98      0.89      0.93      6016
          1       0.40      0.81      0.53       538

avg / total       0.93      0.88      0.90      6554

offer
             precision    recall  f1-score   support

          0       1.00      0.99      1.00      6554
          1       0.00      0.00      0.00         0

avg / total       1.00      0.99      1.00      6554

aid_related
             precision    recall  f1-score   support

          0       0.88      0.71      0.78      4767
          1       0.49      0.75      0.59      1787

avg / total       0.78      0.72      0.73      6554

medical_help
             precision    recall  f1-sco

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [29]:
def build_model():
    pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf',MultiOutputClassifier(RandomForestClassifier()),
        )
    ])

    parameters = {
        'clf__estimator__n_estimators': [10, 20],
       'clf__estimator__min_samples_split': [2, 4]
    }
    
    cv = GridSearchCV(pipeline, param_grid=parameters)

    return cv

In [ ]:
#    parameters = {
#        'estimator__vect__ngram_range': ((1, 1), (1, 2)),
#        'estimator__vect__max_df': (0.5, 1.0),
#        'estimator__vect__max_features': (None, 5000, 10000),
#        'estimator__tfidf__use_idf': (True, False),
#        'estimator__clf__estimator__n_estimators': [50, 100, 200],
#       'estimator__clf__estimator__min_samples_split': [2, 3, 4]
#    }

In [73]:
#parameters = {
        #'vect__ngram_range': ((1, 1), (1, 2)),
        #'vect__max_df': (0.5, 1.0),
        #'vect__max_features': (None, 5000, 10000),
       # 'tfidf__use_idf': (True, False),
       # 'clf__n_estimators': [50, 100, 200],
#       'clf__min_samples_split': [2, 3, 4]
#    }
#param_grid=parameters

#def cv():
#    cv = GridSearchCV(pipeline,param_grid=parameters)
#    return cv

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [31]:
model = build_model()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

In [32]:
for i in range(len(Y.columns)):
    print(Y.columns[i])
    print(classification_report(y_pred[:,i], y_test.iloc[:,i]))

related
             precision    recall  f1-score   support

          0       0.30      0.68      0.41       675
          1       0.95      0.81      0.87      5870
          2       0.09      0.56      0.15         9

avg / total       0.89      0.79      0.83      6554

request
             precision    recall  f1-score   support

          0       0.99      0.89      0.94      6021
          1       0.42      0.86      0.56       533

avg / total       0.94      0.89      0.91      6554

offer
             precision    recall  f1-score   support

          0       1.00      0.99      1.00      6554
          1       0.00      0.00      0.00         0

avg / total       1.00      0.99      1.00      6554

aid_related
             precision    recall  f1-score   support

          0       0.89      0.73      0.80      4677
          1       0.53      0.78      0.63      1877

avg / total       0.79      0.74      0.75      6554

medical_help
             precision    recall  f1-sco

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [28]:
#pipeline.get_params().keys()

dict_keys(['memory', 'steps', 'vect', 'tfidf', 'clf', 'vect__analyzer', 'vect__binary', 'vect__decode_error', 'vect__dtype', 'vect__encoding', 'vect__input', 'vect__lowercase', 'vect__max_df', 'vect__max_features', 'vect__min_df', 'vect__ngram_range', 'vect__preprocessor', 'vect__stop_words', 'vect__strip_accents', 'vect__token_pattern', 'vect__tokenizer', 'vect__vocabulary', 'tfidf__norm', 'tfidf__smooth_idf', 'tfidf__sublinear_tf', 'tfidf__use_idf', 'clf__estimator__bootstrap', 'clf__estimator__class_weight', 'clf__estimator__criterion', 'clf__estimator__max_depth', 'clf__estimator__max_features', 'clf__estimator__max_leaf_nodes', 'clf__estimator__min_impurity_decrease', 'clf__estimator__min_impurity_split', 'clf__estimator__min_samples_leaf', 'clf__estimator__min_samples_split', 'clf__estimator__min_weight_fraction_leaf', 'clf__estimator__n_estimators', 'clf__estimator__n_jobs', 'clf__estimator__oob_score', 'clf__estimator__random_state', 'clf__estimator__verbose', 'clf__estimator__

In [24]:
#model.get_params().keys()

dict_keys(['cv', 'error_score', 'estimator__memory', 'estimator__steps', 'estimator__vect', 'estimator__tfidf', 'estimator__clf', 'estimator__vect__analyzer', 'estimator__vect__binary', 'estimator__vect__decode_error', 'estimator__vect__dtype', 'estimator__vect__encoding', 'estimator__vect__input', 'estimator__vect__lowercase', 'estimator__vect__max_df', 'estimator__vect__max_features', 'estimator__vect__min_df', 'estimator__vect__ngram_range', 'estimator__vect__preprocessor', 'estimator__vect__stop_words', 'estimator__vect__strip_accents', 'estimator__vect__token_pattern', 'estimator__vect__tokenizer', 'estimator__vect__vocabulary', 'estimator__tfidf__norm', 'estimator__tfidf__smooth_idf', 'estimator__tfidf__sublinear_tf', 'estimator__tfidf__use_idf', 'estimator__clf__estimator__bootstrap', 'estimator__clf__estimator__class_weight', 'estimator__clf__estimator__criterion', 'estimator__clf__estimator__max_depth', 'estimator__clf__estimator__max_features', 'estimator__clf__estimator__max

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [47]:
# Try AdaBoost
def build_model_2():
    pipeline1 = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf',MultiOutputClassifier(AdaBoostClassifier()),
        )
    ])

    parameters = {
        'clf__estimator__n_estimators': [10, 20],
        #'clf__estimator__min_samples_leaf': [2, 4]
       #'clf__estimator__min_samples_split': [2, 4]
    }
    
    cv = GridSearchCV(pipeline1, param_grid=parameters)

    return cv

In [48]:
model = build_model_2()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

In [49]:
for i in range(len(Y.columns)):
    print(Y.columns[i])
    print(classification_report(y_pred[:,i], y_test.iloc[:,i]))

related
             precision    recall  f1-score   support

          0       0.20      0.55      0.29       548
          1       0.95      0.78      0.86      5997
          2       0.07      0.44      0.12         9

avg / total       0.89      0.77      0.81      6554

request
             precision    recall  f1-score   support

          0       0.97      0.91      0.94      5805
          1       0.52      0.77      0.62       749

avg / total       0.92      0.89      0.90      6554

offer
             precision    recall  f1-score   support

          0       1.00      0.99      1.00      6548
          1       0.00      0.00      0.00         6

avg / total       1.00      0.99      1.00      6554

aid_related
             precision    recall  f1-score   support

          0       0.87      0.73      0.79      4575
          1       0.54      0.75      0.63      1979

avg / total       0.77      0.74      0.74      6554

medical_help
             precision    recall  f1-sco

In [52]:
# Try Other features
def build_model_3():
    pipeline1 = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf',MultiOutputClassifier(RandomForestClassifier()),
        )
    ])

    parameters = {
        'clf__estimator__n_estimators': [10, 20],
        'clf__estimator__min_samples_leaf': [2, 4],
        'clf__estimator__min_samples_split': [2, 4]
    }
    
    cv = GridSearchCV(pipeline1, param_grid=parameters)

    return cv

In [53]:
model = build_model_3()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

In [54]:
for i in range(len(Y.columns)):
    print(Y.columns[i])
    print(classification_report(y_pred[:,i], y_test.iloc[:,i]))

related
             precision    recall  f1-score   support

          0       0.19      0.75      0.31       398
          1       0.98      0.79      0.87      6150
          2       0.07      0.67      0.13         6

avg / total       0.93      0.79      0.84      6554

request
             precision    recall  f1-score   support

          0       0.99      0.87      0.93      6219
          1       0.27      0.90      0.42       335

avg / total       0.96      0.87      0.90      6554

offer
             precision    recall  f1-score   support

          0       1.00      0.99      1.00      6554
          1       0.00      0.00      0.00         0

avg / total       1.00      0.99      1.00      6554

aid_related
             precision    recall  f1-score   support

          0       0.88      0.75      0.81      4482
          1       0.59      0.78      0.67      2072

avg / total       0.79      0.76      0.77      6554

medical_help
             precision    recall  f1-sco

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### 9. Export your model as a pickle file

In [57]:
import pickle
 
f=open('model.pkl','wb')
pickle.dump(model,f)
f.close

<function BufferedWriter.close>

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.